In [ ]:
# -*- coding: utf-8 -*-
"""
Created on May 12, 2024
Last modified on May 14, 2024
@Author: Guan-Fu Liu

To test the MassLifetime.py
"""
import numpy as np
import pandas as pd
from scipy.integrate import quad
import sys
sys.path.insert(0, '/Users/liuguanfu/Workspace/SAS-21/targets/MRK1216/spex/jupyter/IMF/chemevoimf')
from MassLifetime import MassLifetime
import constants
import utils
import h5py
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
%matplotlib widget

In [ ]:
mass_lifetime_file = "./inputs/NuPyCEE/agb_and_massive_stars_C15_N13_0_5_HNe/yields2.h5"
file = h5py.File(mass_lifetime_file, "r")
# file = h5py.File("../inputs/NuPyCEE/isotope_yield_table_portinari98_marigo01_gce_totalyields/yields1.h5", "r")
lifetime = { }
for key in file.keys():
    lifetime[key] = { }
    lifetime[key]['Mini'] = file[key]['MassLifetime'][:,0].astype(float)
    lifetime[key]['lifetime'] = file[key]['MassLifetime'][:,1].astype(float)

mass_lifetime = MassLifetime(lifetime=lifetime)

mass_lifetime = MassLifetime()

In [ ]:
Z_list = list(mass_lifetime.lifetime.keys())
ms = np.logspace(np.log10(0.08), np.log10(150), 100)
t = { }
for Z in Z_list:
    t[Z] = mass_lifetime.mass_to_lifetime(ms, float(Z.replace("Z=","")), kind='nearest')

fig, ax = plt.subplots()
for Z in Z_list:
    ax.plot(ms, t[Z], label=Z)
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend()

In [ ]:
m = 20
Zs = np.logspace(-4, np.log10(0.08), 1000)
kinds = ['linear-linear', 'linear-log', 'log-linear', 'log-log', 'nearest']
t = { }

for kind in kinds:
    t[kind] = np.array([mass_lifetime.mass_to_lifetime(m, Z, kind=kind)
                for Z in Zs])

fig, ax = plt.subplots()
for kind in kinds:
    ax.plot(Zs, t[kind], label=kind)
ax.set_xscale('log')
# ax.set_yscale('log')
ax.set_xlabel('Z')
ax.set_ylabel('t')
ax.legend()

In [ ]:
t = 1e7
Zs = np.logspace(-4, np.log10(0.08), 1000)
kinds = ['linear-linear', 'linear-log', 'log-linear', 'log-log', 'nearest']
m = { }

for kind in kinds:
    m[kind] = np.array([mass_lifetime.lifetime_to_mass(t, Z, kind=kind)
                for Z in Zs])

fig, ax = plt.subplots()
for kind in kinds:
    ax.plot(Zs, m[kind], label=kind)
ax.set_xscale('log')
ax.set_xlabel('Z')
ax.set_ylabel('m')
ax.legend()

In [ ]:
Zs = np.logspace(-4, np.log10(0.08), 100)
ms = np.logspace(np.log10(0.08), np.log10(150), 100)
kind = 'linear-linear'
Zs, ms = np.meshgrid(Zs, ms)
t = np.array([mass_lifetime.mass_to_lifetime(m, Z, kind)
              for m, Z in zip(ms.ravel(), Zs.ravel())]).reshape(ms.shape)
fig, ax = plt.subplots()
c = ax.pcolormesh(Zs, ms, t, norm=colors.LogNorm(vmin=t.min(), vmax=t.max()), cmap = cm.coolwarm)
cbar = fig.colorbar(c, ax=ax)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Z')
ax.set_ylabel(r'M [$M_{\odot}$]')
ax.set_title(kind)
cbar.set_label('t [yr]')
plt.show()

In [ ]:
Zs = np.logspace(-4, np.log10(0.08), 100)
ms = np.logspace(np.log10(0.08), np.log10(150), 100)
kind = 'linear-log'
Zs, ms = np.meshgrid(Zs, ms)
t = np.array([mass_lifetime.mass_to_lifetime(m, Z, kind)
              for m, Z in zip(ms.ravel(), Zs.ravel())]).reshape(ms.shape)
fig, ax = plt.subplots()
c = ax.pcolormesh(Zs, ms, t, norm=colors.LogNorm(vmin=t.min(), vmax=t.max()), cmap = cm.coolwarm)
cbar = fig.colorbar(c, ax=ax)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Z')
ax.set_ylabel(r'M [$M_{\odot}$]')
ax.set_title(kind)
cbar.set_label('t [yr]')
plt.show()

In [ ]:
file = h5py.File("../inputs/NuPyCEE/isotope_yield_table_portinari98_marigo01_gce_totalyields/yields1.h5", "r")
lifetime = { }
for key in file.keys():
    lifetime[key] = { }
    lifetime[key]['Mini'] = file[key]['MassLifetime'][:,0].astype(float)
    lifetime[key]['lifetime'] = file[key]['MassLifetime'][:,1].astype(float)

mass_lifetime = MassLifetime(lifetime=lifetime)

In [ ]:
mass_lifetime.lifetime_to_mass(1e10, 0)

In [ ]:
import math

file = open('../../galIMF/yield_tables/rearranged/setllar_lifetime_from_portinari98/portinari98_Z=0.0004.txt','r')
lines = file.readlines()
file.close()
Mini1 = np.array(lines[3].split(), dtype=float)
lifetime1 = np.array(lines[5].split(), dtype=float)

def fucntion_mass_boundary(time, mass_grid_for_lifetime, lifetime):
    # The adopted spline fit of portinari98 lifetime is not monotonic at the massive end. 
    # But this function ensures that lifetime is monotonically smaller for more massive stars.
    mass = mass_grid_for_lifetime
    length_list_lifetime = len(lifetime)
    x = round(length_list_lifetime / 2)
    loop_number_fucntion_mass_boundary = math.ceil(math.log(length_list_lifetime, 2))
    mass_boundary = 10000
    if lifetime[x] == time:
        mass_boundary = mass[x]
    else:
        i = 0
        low = 0
        high = length_list_lifetime
        while i < loop_number_fucntion_mass_boundary:
            if lifetime[x] > time:
                low = x
                x = x + round((high - x) / 2)
            else:
                high = x
                x = x - round((x - low) / 2)
            (i) = (i + 1)
        if x == length_list_lifetime - 1:
            mass_boundary = mass[x]
        else:
            if lifetime[x - 1] > time > lifetime[x]:
                x = x - 1
            mass_boundary = round(mass[x] + (mass[x + 1] - mass[x]) * (lifetime[x] - time) / (
                lifetime[x] - lifetime[x + 1]), 5)
    return mass_boundary


for i in range(1,10):
    print("After %0.2e yr"%(i*1e7))
    print("Stars with mass >=%0.4f Msun will explode in ChemEvoIMF"%utils.lifetime_to_mass(i*1e7, 0))
    print("Stars with mass >=%0.4f Msun will explode in galIMF"%fucntion_mass_boundary(i*1e7, Mini1, lifetime1))
    print("Stars with mass >=%0.4f Msun will explode in new mass lifetime table"%mass_lifetime.lifetime_to_mass(i*1e7, 0))
